In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import joblib
from sklearn.model_selection import TimeSeriesSplit
import optuna 
from sklearn.model_selection import cross_val_score


c:\Users\TimPr\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def clean_column_names(df):
    """Clean column names to be compatible with XGBoost"""
    df = df.copy()
    # Remove brackets and clean special characters
    df.columns = (df.columns
                 .str.replace('[', '')
                 .str.replace(']', '')
                 .str.replace(' ', '_')
                 .str.replace('(', '')
                 .str.replace(')', '')
                 .str.replace('ö', 'oe')  # Handle German special characters
                 .str.replace('ä', 'ae')
                 .str.replace('ü', 'ue'))
    return df

def create_lagged_features(df, target_col, lag_hours=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 24, 48, 72, 168, 336, 672, 1008, 2016, 8760]):
    """Create additional lagged features for time series data"""
    df_copy = df.copy()
    
    # Create lags for a variety of time periods (e.g., 1h, 2h, 1 day, 1 week, etc.)
    for lag in lag_hours:
        df_copy[f'{target_col}_lag_{lag}h'] = df_copy[target_col].shift(lag)
        
    # Drop rows with NaN values created by lagging
    df_copy = df_copy.dropna()
    
    return df_copy


def preprocess_load_data(data, split_date='2023-09-30'):
    """Preprocess the load data including lagged features"""
    # Create a copy of the data
    df = data.copy()
    
    # Store the original load column name
    original_load_col = 'Gesamt (Netzlast) [MWh] Berechnete Auflösungen'
    
    # Convert Date column to datetime and sort
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    
    # Create lagged features before cleaning column names
    df = create_lagged_features(df, target_col=original_load_col)
    
    # Clean column names
    df = clean_column_names(df)
    
    # Get the cleaned load column name
    load_col = 'Gesamt_Netzlast_MWh_Berechnete_Aufloesungen'
    
    # Rename to 'load'
    df = df.rename(columns={load_col: 'load'})
    
    # Select features for the model (using cleaned names)
    base_features = [
        'hour', 'dayofyear_cos', 'dayofweek', 'dayofweek_sin',
        'is_workday', 'hour_cos', 'date_offset', 'dayofyear',
        'Kernenergie_MWh_Berechnete_Aufloesungen',
        'Steinkohle_MWh_Berechnete_Aufloesungen',
        'Holiday_Not_a_Holiday', 'hour_sin',
        'Wind_Onshore_MWh_Berechnete_Aufloesungen'
    ]
    
    # Add lagged feature names
    lag_features = [col for col in df.columns if 'lag' in col]
    feature_columns = base_features + lag_features
    
    # Split data based on date
    train_data = df[df['Date'] < split_date]
    test_data = df[df['Date'] >= split_date]
    
    # Split features and target
    X_train = train_data[feature_columns]
    y_train = train_data['load']
    X_test = test_data[feature_columns]
    y_test = test_data['load']
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert back to pandas DataFrames
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_columns)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_columns)
    
    print(f"\nTraining data shape: {X_train_scaled.shape}")
    print(f"Testing data shape: {X_test_scaled.shape}")
    print(f"Training period: {train_data['Date'].min()} to {train_data['Date'].max()}")
    print(f"Testing period: {test_data['Date'].min()} to {test_data['Date'].max()}")
    
    return X_train_scaled, X_test_scaled, y_train, y_test, feature_columns

In [28]:
def evaluate_models_cv(X_train, y_train, n_folds=5):
    """Evaluate multiple models using cross-validation on training data"""
    
    # Define base models with default parameters
    models = {
        'XGBoost': xgb.XGBRegressor(
            n_estimators=2000,
            learning_rate=0.01,
            max_depth=20,
            random_state=42
        ),
        'LightGBM': lgb.LGBMRegressor(
            n_estimators=4000,
            learning_rate=0.001,
            max_depth=500,
            random_state=42
        ),
        'RandomForest': RandomForestRegressor(
            n_estimators=100,
            max_depth=20,
            random_state=42
        )
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"\nEvaluating {name} using {n_folds}-fold cross-validation...")
        
        # Perform cross-validation
        scores = cross_val_score(
            model,
            X_train,
            y_train,
            cv=n_folds,
            scoring='neg_mean_absolute_percentage_error',
            n_jobs=-1  # Use all available cores
        )
        
        # Convert negative MAPE scores to positive percentages
        mape_scores = -scores * 100
        
        # Store results
        results[name] = {
            'cv_scores': mape_scores,
            'mean_cv_mape': mape_scores.mean(),
            'std_cv_mape': mape_scores.std()
        }
        
        print(f"{name} Results:")
        print(f"Mean CV MAPE: {mape_scores.mean():.2f}% (±{mape_scores.std():.2f})")
        print(f"Individual fold MAPEs: {[f'{score:.2f}%' for score in mape_scores]}")
    
    return results

In [31]:
# Example usage:
# Load the data
df = pd.read_csv('../Data/selected_features.csv')

# Preprocess the data
X_train, X_test, y_train, y_test, feature_names = preprocess_load_data(df)

# Train and evaluate models
results = evaluate_models_cv(X_train, y_train)


Training data shape: (34319, 34)
Testing data shape: (9541, 34)
Training period: 2019-10-31 18:00:00 to 2023-09-29 23:00:00
Testing period: 2023-09-30 00:00:00 to 2024-10-30 23:00:00

Evaluating XGBoost using 5-fold cross-validation...
XGBoost Results:
Mean CV MAPE: 0.86% (±0.12)
Individual fold MAPEs: ['0.83%', '0.81%', '0.77%', '0.81%', '1.11%']

Evaluating LightGBM using 5-fold cross-validation...
LightGBM Results:
Mean CV MAPE: 1.09% (±0.13)
Individual fold MAPEs: ['1.05%', '1.05%', '1.02%', '1.00%', '1.34%']

Evaluating RandomForest using 5-fold cross-validation...
RandomForest Results:
Mean CV MAPE: 0.89% (±0.10)
Individual fold MAPEs: ['0.86%', '0.87%', '0.79%', '0.82%', '1.09%']


## Specific model: 

In [ ]:
def clean_column_names(df):
    """Clean column names to be compatible with XGBoost"""
    df = df.copy()
    # Remove brackets and clean special characters
    df.columns = (df.columns
                 .str.replace('[', '')
                 .str.replace(']', '')
                 .str.replace(' ', '_')
                 .str.replace('(', '')
                 .str.replace(')', '')
                 .str.replace('ö', 'oe')  # Handle German special characters
                 .str.replace('ä', 'ae')
                 .str.replace('ü', 'ue'))
    return df

def create_lagged_features(df, target_col, lag_hours=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 24, 48, 72, 168, 336, 672, 1008, 2016, 8760]):
    """Create additional lagged features for time series data"""
    df_copy = df.copy()
    
    # Create lags for a variety of time periods (e.g., 1h, 2h, 1 day, 1 week, etc.)
    for lag in lag_hours:
        df_copy[f'{target_col}_lag_{lag}h'] = df_copy[target_col].shift(lag)
        
    # Drop rows with NaN values created by lagging
    df_copy = df_copy.dropna()
    
    return df_copy


def preprocess_load_data(data, split_date='2023-09-30'):
    """Preprocess the load data including lagged features"""
    # Create a copy of the data
    df = data.copy()
    
    # Store the original load column name
    original_load_col = 'Gesamt (Netzlast) [MWh] Berechnete Auflösungen'
    
    # Convert Date column to datetime and sort
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')
    
    # Create lagged features before cleaning column names
    df = create_lagged_features(df, target_col=original_load_col)
    
    # Clean column names
    df = clean_column_names(df)
    
    # Get the cleaned load column name
    load_col = 'Gesamt_Netzlast_MWh_Berechnete_Aufloesungen'
    
    # Rename to 'load'
    df = df.rename(columns={load_col: 'load'})
    
    # Select features for the model (using cleaned names)
    base_features = [
        'hour', 'dayofyear_cos', 'dayofweek', 'dayofweek_sin',
        'is_workday', 'hour_cos', 'date_offset', 'dayofyear',
        'Kernenergie_MWh_Berechnete_Aufloesungen',
        'Steinkohle_MWh_Berechnete_Aufloesungen',
        'Holiday_Not_a_Holiday', 'hour_sin',
        'Wind_Onshore_MWh_Berechnete_Aufloesungen'
    ]
    
    # Add lagged feature names
    lag_features = [col for col in df.columns if 'lag' in col]
    feature_columns = base_features + lag_features
    
    # Split data based on date
    train_data = df[df['Date'] < split_date]
    test_data = df[df['Date'] >= split_date]
    
    # Split features and target
    X_train = train_data[feature_columns]
    y_train = train_data['load']
    X_test = test_data[feature_columns]
    y_test = test_data['load']
    
    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert back to pandas DataFrames
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_columns)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_columns)
    
    print(f"\nTraining data shape: {X_train_scaled.shape}")
    print(f"Testing data shape: {X_test_scaled.shape}")
    print(f"Training period: {train_data['Date'].min()} to {train_data['Date'].max()}")
    print(f"Testing period: {test_data['Date'].min()} to {test_data['Date'].max()}")
    
    return X_train_scaled, X_test_scaled, y_train, y_test, feature_columns

def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    """Train and evaluate multiple models"""
    models = {
        'XGBoost': xgb.XGBRegressor(
            n_estimators=1000,
            learning_rate=0.001,
            max_depth=30,
            random_state=42
        ),
        'LightGBM': lgb.LGBMRegressor(
            n_estimators=1000,
            learning_rate=0.001,
            max_depth=30,
            random_state=42
        ),
        'RandomForest': RandomForestRegressor(
            n_estimators=100,
            max_depth=10,
            random_state=42
        )
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"\nTraining {name}...")
        model.fit(X_train, y_train)
        
        # Make predictions
        train_pred = model.predict(X_train)
        test_pred = model.predict(X_test)
        
        # Calculate MAPE
        train_mape = mean_absolute_percentage_error(y_train, train_pred) * 100
        test_mape = mean_absolute_percentage_error(y_test, test_pred) * 100
        
        results[name] = {
            'train_mape': train_mape,
            'test_mape': test_mape,
            'model': model
        }
        
        print(f"{name} Results:")
        print(f"Train MAPE: {train_mape:.2f}%")
        print(f"Test MAPE: {test_mape:.2f}%")
    
    return results

In [ ]:

# Train and evaluate models
results = train_and_evaluate_models(X_train, X_test, y_train, y_test)

In [6]:
# Save models
joblib.dump(results['XGBoost']['model'], 'xgboost_model.pkl')
joblib.dump(results['LightGBM']['model'], 'lightgbm_model.pkl')
joblib.dump(results['RandomForest']['model'], 'randomforest_model.pkl')

# # To load the models later:
# xgboost_model = joblib.load('xgboost_model.pkl')
# lightgbm_model = joblib.load('lightgbm_model.pkl')
# randomforest_model = joblib.load('randomforest_model.pkl')


['randomforest_model.pkl']

In [36]:

# Make predictions on the test set
xgboost_pred = results['XGBoost']['model'].predict(X_test)
lightgbm_pred = results['LightGBM']['model'].predict(X_test)
randomforest_pred = results['RandomForest']['model'].predict(X_test)

# Create subplots
fig, axes = plt.subplots(3, 1, figsize=(12, 18), sharex=True)

# XGBoost plot
axes[0].plot(y_test.index, y_test, label='Actual Load', color='blue')
axes[0].plot(y_test.index, xgboost_pred, label='XGBoost Forecast', color='green', linestyle='--')
axes[0].set_title('XGBoost: Actual vs Forecasted Load')
axes[0].set_ylabel('Load [MWh]')
axes[0].legend()

# LightGBM plot
axes[1].plot(y_test.index, y_test, label='Actual Load', color='blue')
axes[1].plot(y_test.index, lightgbm_pred, label='LightGBM Forecast', color='red', linestyle='--')
axes[1].set_title('LightGBM: Actual vs Forecasted Load')
axes[1].set_ylabel('Load [MWh]')
axes[1].legend()

# RandomForest plot
axes[2].plot(y_test.index, y_test, label='Actual Load', color='blue')
axes[2].plot(y_test.index, randomforest_pred, label='RandomForest Forecast', color='orange', linestyle='--')
axes[2].set_title('RandomForest: Actual vs Forecasted Load')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Load [MWh]')
axes[2].legend()

# Display the plots
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


KeyError: 'model'

In [35]:
# Define the range for the week to zoom in on (adjust as needed)
week_start = '2023-10-01'
week_end = '2023-10-07'

# Filter the data for the specified week
y_test_week = y_test.loc[week_start:week_end]
xgboost_pred_week = pd.Series(xgboost_pred, index=y_test.index).loc[week_start:week_end]
lightgbm_pred_week = pd.Series(lightgbm_pred, index=y_test.index).loc[week_start:week_end]
randomforest_pred_week = pd.Series(randomforest_pred, index=y_test.index).loc[week_start:week_end]

# Create subplots for the zoomed-in week
fig, axes = plt.subplots(3, 1, figsize=(12, 18), sharex=True)

# XGBoost plot
axes[0].plot(y_test_week.index, y_test_week, label='Actual Load', color='blue')
axes[0].plot(y_test_week.index, xgboost_pred_week, label='XGBoost Forecast', color='green', linestyle='--')
axes[0].set_title('XGBoost: Actual vs Forecasted Load (Zoomed-In Week)')
axes[0].set_ylabel('Load [MWh]')
axes[0].legend()

# LightGBM plot
axes[1].plot(y_test_week.index, y_test_week, label='Actual Load', color='blue')
axes[1].plot(y_test_week.index, lightgbm_pred_week, label='LightGBM Forecast', color='red', linestyle='--')
axes[1].set_title('LightGBM: Actual vs Forecasted Load (Zoomed-In Week)')
axes[1].set_ylabel('Load [MWh]')
axes[1].legend()

# RandomForest plot
axes[2].plot(y_test_week.index, y_test_week, label='Actual Load', color='blue')
axes[2].plot(y_test_week.index, randomforest_pred_week, label='RandomForest Forecast', color='orange', linestyle='--')
axes[2].set_title('RandomForest: Actual vs Forecasted Load (Zoomed-In Week)')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Load [MWh]')
axes[2].legend()

# Display the zoomed-in plots
plt.xticks


KeyError: '2023-10-01'

## Test CV Hyperparametersearch:

In [37]:
from tqdm.auto import tqdm
from sklearn.model_selection import KFold


In [43]:
def test_optimize_and_evaluate_models_cv(X_train, y_train, n_folds=5, n_trials=10):
    """Quick test for optimizing and evaluating models using cross-validation"""
    
    def objective_xgb(trial, X, y):
        params = {
            'n_estimators': trial.suggest_categorical('n_estimators', [500, 1000, 2000, 4000]),
            'max_depth': trial.suggest_int('max_depth', 10, 30),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
            'random_state': 42
        }
        model = xgb.XGBRegressor(**params)
        scores = cross_val_score(
            model, X, y,
            cv=n_folds,
            scoring='neg_mean_absolute_percentage_error',
            n_jobs=-1
        )
        return -scores.mean() * 100

    def objective_lgb(trial, X, y):
        params = {
            'n_estimators': trial.suggest_categorical('n_estimators', [500, 1000, 2000, 4000]),
            'max_depth': trial.suggest_int('max_depth', 10, 30),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
            'random_state': 42
        }
        model = lgb.LGBMRegressor(**params)
        scores = cross_val_score(
            model, X, y,
            cv=n_folds,
            scoring='neg_mean_absolute_percentage_error',
            n_jobs=-1
        )
        return -scores.mean() * 100

    results = {}
    objectives = {
        'XGBoost': objective_xgb,
        'LightGBM': objective_lgb,
    }

    for name, objective in objectives.items():
        print(f"\nTesting {name} optimization...")
        study = optuna.create_study(direction='minimize')
        study.optimize(
            lambda trial: objective(trial, X_train, y_train),
            n_trials=n_trials,
            n_jobs=-1
        )
        
        best_params = study.best_params
        print(f"\n{name} Best parameters (test): {best_params}")
        
        # Test evaluation
        print(f"\nEvaluating best {name} model (test)...")
        if name == 'XGBoost':
            best_model = xgb.XGBRegressor(**best_params)
        elif name == 'LightGBM':
            best_model = lgb.LGBMRegressor(**best_params)
        
        scores = cross_val_score(
            best_model,
            X_train,
            y_train,
            cv=n_folds,
            scoring='neg_mean_absolute_percentage_error',
            n_jobs=-1
        )
        mape_scores = -scores * 100
        print(f"{name} Test Results:")
        print(f"Mean CV MAPE: {mape_scores.mean():.2f}%")
        print(f"Std CV MAPE: {mape_scores.std():.2f}%")
        results[name] = mape_scores.mean()

    return results

# Usage:
# test_results = test_optimize_and_evaluate_models_cv(X_train, y_train, n_folds=3, n_trials=3)


In [42]:
test_results = test_optimize_and_evaluate_models_cv(X_train, y_train)

[I 2025-01-14 11:38:52,641] A new study created in memory with name: no-name-ec643319-2f0b-4e01-8019-5ecac19b95e1



Testing XGBoost optimization...


C:\Users\TimPr\AppData\Local\Temp\ipykernel_14884\2037188567.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
